In [3]:
import json
import csv
#import tweepy
import re
import nltk
from nltk.stem import WordNetLemmatizer
import pandas as pd
#import GetOldTweets3 as got
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
import numpy as np

In [4]:
df=pd.read_csv('../../Datasets/Quora/IITBQuora.csv')


In [5]:
df

,Unnamed: 0,Topic,question,Followers,Answers,Ratio,Engagement potential,Last action,Parsed time,Question link,answers
0,0,Indian-Institute-of-Technology-Bombay-IITB,Do only the students who start preparation for...,5,1,5/1,Yes,Followed,2020-07-22,/Do-only-the-students-who-start-preparation-fo...,Well actually not….though I am a NEET aspirant...
1,1,Indian-Institute-of-Technology-Bombay-IITB,How could I get into IIT Bombay? Is it possibl...,1,1,1/1,No,Followed,2020-07-21,/How-could-I-get-into-IIT-Bombay-Is-it-possibl...,"2 months and IIT Bombay, well umm what to say ..."
2,2,Indian-Institute-of-Technology-Bombay-IITB,Can Kalpit Veerwal stop begging for ratings in...,15,1,15/1,Yes,Followed,2020-07-21,/Can-Kalpit-Veerwal-stop-begging-for-ratings-i...,"Hi, Nobody except Kalpit should be worried abo..."
3,3,Indian-Institute-of-Technology-Bombay-IITB,What are the subjects in aerospace engineering...,2,1,2/1,No,Followed,2020-07-20,/What-are-the-subjects-in-aerospace-engineerin...,"For such quarries in future as well, please vi..."
4,4,Indian-Institute-of-Technology-Bombay-IITB,What is the full form of KGP?,2,1,2/1,No,Followed,2020-07-20,/What-is-the-full-form-of-KGP,"Knibb, Gormezano, & Partners Kannada Ganaka Pa..."
...,...,...,...,...,...,...,...,...,...,...,...
766,766,Indian-Institute-of-Technology-Bombay-IITB,What is the procedure for registration for Moo...,1,1,1/1,No,Followed,2019-09-30,/What-is-the-procedure-for-registration-for-Mo...,"Go to Mood Indigo, IIT Bombay | Asia's Largest..."
767,767,Indian-Institute-of-Technology-Bombay-IITB,Is it possible to participate in the Techfest ...,4,1,4/1,Yes,Followed,2019-09-29,/Is-it-possible-to-participate-in-the-Techfest...,"Yes, Techfest conducts zonals to increase its ..."
768,768,Indian-Institute-of-Technology-Bombay-IITB,What is the eligibility for participating in T...,1,1,1/1,No,Followed,2019-09-27,/What-is-the-eligibility-for-participating-in-...,To participate in IIT Bombay Techfest Robowars...
769,769,Indian-Institute-of-Technology-Bombay-IITB,What is the closing and opening rank of IIT Ka...,1,1,1/1,No,Followed,2019-09-25,/What-is-the-closing-and-opening-rank-of-IIT-K...,Search on Google www.jossa.ac.in


In [6]:
df=df.reset_index()


In [8]:
df=df.drop_duplicates()

In [9]:
df=df.drop(['index'],axis=1)

In [12]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
df['compound'] = [analyzer.polarity_scores(v)['compound'] for v in df['answers']]

In [13]:
#classifying sentiments as positive,negative and neutral
def sentiment(compound):
    res = ""
    if(compound >= 0.05):
        res="Positive"
    elif(compound <= - 0.05):
        res="Negative"
    else:
        res="Neutral"
    return res

df['Sentiment'] = df['compound'].apply(lambda x: sentiment(x))

In [14]:
df

,Unnamed: 0,Topic,question,Followers,Answers,Ratio,Engagement potential,Last action,Parsed time,Question link,answers,compound,Sentiment
0,0,Indian-Institute-of-Technology-Bombay-IITB,Do only the students who start preparation for...,5,1,5/1,Yes,Followed,2020-07-22,/Do-only-the-students-who-start-preparation-fo...,Well actually not….though I am a NEET aspirant...,0.8388,Positive
1,1,Indian-Institute-of-Technology-Bombay-IITB,How could I get into IIT Bombay? Is it possibl...,1,1,1/1,No,Followed,2020-07-21,/How-could-I-get-into-IIT-Bombay-Is-it-possibl...,"2 months and IIT Bombay, well umm what to say ...",0.5423,Positive
2,2,Indian-Institute-of-Technology-Bombay-IITB,Can Kalpit Veerwal stop begging for ratings in...,15,1,15/1,Yes,Followed,2020-07-21,/Can-Kalpit-Veerwal-stop-begging-for-ratings-i...,"Hi, Nobody except Kalpit should be worried abo...",-0.3164,Negative
3,3,Indian-Institute-of-Technology-Bombay-IITB,What are the subjects in aerospace engineering...,2,1,2/1,No,Followed,2020-07-20,/What-are-the-subjects-in-aerospace-engineerin...,"For such quarries in future as well, please vi...",0.7351,Positive
4,4,Indian-Institute-of-Technology-Bombay-IITB,What is the full form of KGP?,2,1,2/1,No,Followed,2020-07-20,/What-is-the-full-form-of-KGP,"Knibb, Gormezano, & Partners Kannada Ganaka Pa...",-0.5423,Negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...
766,766,Indian-Institute-of-Technology-Bombay-IITB,What is the procedure for registration for Moo...,1,1,1/1,No,Followed,2019-09-30,/What-is-the-procedure-for-registration-for-Mo...,"Go to Mood Indigo, IIT Bombay | Asia's Largest...",0.5411,Positive
767,767,Indian-Institute-of-Technology-Bombay-IITB,Is it possible to participate in the Techfest ...,4,1,4/1,Yes,Followed,2019-09-29,/Is-it-possible-to-participate-in-the-Techfest...,"Yes, Techfest conducts zonals to increase its ...",0.8679,Positive
768,768,Indian-Institute-of-Technology-Bombay-IITB,What is the eligibility for participating in T...,1,1,1/1,No,Followed,2019-09-27,/What-is-the-eligibility-for-participating-in-...,To participate in IIT Bombay Techfest Robowars...,0.8271,Positive
769,769,Indian-Institute-of-Technology-Bombay-IITB,What is the closing and opening rank of IIT Ka...,1,1,1/1,No,Followed,2019-09-25,/What-is-the-closing-and-opening-rank-of-IIT-K...,Search on Google www.jossa.ac.in,0.0000,Neutral


In [15]:
df['score'] = pd.cut(df['compound'], bins=5, labels=[1, 2, 3, 4, 5])
df

,Unnamed: 0,Topic,question,Followers,Answers,Ratio,Engagement potential,Last action,Parsed time,Question link,answers,compound,Sentiment,score
0,0,Indian-Institute-of-Technology-Bombay-IITB,Do only the students who start preparation for...,5,1,5/1,Yes,Followed,2020-07-22,/Do-only-the-students-who-start-preparation-fo...,Well actually not….though I am a NEET aspirant...,0.8388,Positive,5
1,1,Indian-Institute-of-Technology-Bombay-IITB,How could I get into IIT Bombay? Is it possibl...,1,1,1/1,No,Followed,2020-07-21,/How-could-I-get-into-IIT-Bombay-Is-it-possibl...,"2 months and IIT Bombay, well umm what to say ...",0.5423,Positive,4
2,2,Indian-Institute-of-Technology-Bombay-IITB,Can Kalpit Veerwal stop begging for ratings in...,15,1,15/1,Yes,Followed,2020-07-21,/Can-Kalpit-Veerwal-stop-begging-for-ratings-i...,"Hi, Nobody except Kalpit should be worried abo...",-0.3164,Negative,2
3,3,Indian-Institute-of-Technology-Bombay-IITB,What are the subjects in aerospace engineering...,2,1,2/1,No,Followed,2020-07-20,/What-are-the-subjects-in-aerospace-engineerin...,"For such quarries in future as well, please vi...",0.7351,Positive,5
4,4,Indian-Institute-of-Technology-Bombay-IITB,What is the full form of KGP?,2,1,2/1,No,Followed,2020-07-20,/What-is-the-full-form-of-KGP,"Knibb, Gormezano, & Partners Kannada Ganaka Pa...",-0.5423,Negative,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
766,766,Indian-Institute-of-Technology-Bombay-IITB,What is the procedure for registration for Moo...,1,1,1/1,No,Followed,2019-09-30,/What-is-the-procedure-for-registration-for-Mo...,"Go to Mood Indigo, IIT Bombay | Asia's Largest...",0.5411,Positive,4
767,767,Indian-Institute-of-Technology-Bombay-IITB,Is it possible to participate in the Techfest ...,4,1,4/1,Yes,Followed,2019-09-29,/Is-it-possible-to-participate-in-the-Techfest...,"Yes, Techfest conducts zonals to increase its ...",0.8679,Positive,5
768,768,Indian-Institute-of-Technology-Bombay-IITB,What is the eligibility for participating in T...,1,1,1/1,No,Followed,2019-09-27,/What-is-the-eligibility-for-participating-in-...,To participate in IIT Bombay Techfest Robowars...,0.8271,Positive,5
769,769,Indian-Institute-of-Technology-Bombay-IITB,What is the closing and opening rank of IIT Ka...,1,1,1/1,No,Followed,2019-09-25,/What-is-the-closing-and-opening-rank-of-IIT-K...,Search on Google www.jossa.ac.in,0.0000,Neutral,3


In [18]:
def normalize(feature):
    max_value = df[feature].max()
    min_value = df[feature].min()
    result = (df[feature] - min_value) / (max_value - min_value)
    return result

In [19]:
df['followers_n']=normalize('Followers')

In [20]:
df

,Unnamed: 0,Topic,question,Followers,Answers,Ratio,Engagement potential,Last action,Parsed time,Question link,answers,compound,Sentiment,score,followers_n
0,0,Indian-Institute-of-Technology-Bombay-IITB,Do only the students who start preparation for...,5,1,5/1,Yes,Followed,2020-07-22,/Do-only-the-students-who-start-preparation-fo...,Well actually not….though I am a NEET aspirant...,0.8388,Positive,5,0.054348
1,1,Indian-Institute-of-Technology-Bombay-IITB,How could I get into IIT Bombay? Is it possibl...,1,1,1/1,No,Followed,2020-07-21,/How-could-I-get-into-IIT-Bombay-Is-it-possibl...,"2 months and IIT Bombay, well umm what to say ...",0.5423,Positive,4,0.010870
2,2,Indian-Institute-of-Technology-Bombay-IITB,Can Kalpit Veerwal stop begging for ratings in...,15,1,15/1,Yes,Followed,2020-07-21,/Can-Kalpit-Veerwal-stop-begging-for-ratings-i...,"Hi, Nobody except Kalpit should be worried abo...",-0.3164,Negative,2,0.163043
3,3,Indian-Institute-of-Technology-Bombay-IITB,What are the subjects in aerospace engineering...,2,1,2/1,No,Followed,2020-07-20,/What-are-the-subjects-in-aerospace-engineerin...,"For such quarries in future as well, please vi...",0.7351,Positive,5,0.021739
4,4,Indian-Institute-of-Technology-Bombay-IITB,What is the full form of KGP?,2,1,2/1,No,Followed,2020-07-20,/What-is-the-full-form-of-KGP,"Knibb, Gormezano, & Partners Kannada Ganaka Pa...",-0.5423,Negative,2,0.021739
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
766,766,Indian-Institute-of-Technology-Bombay-IITB,What is the procedure for registration for Moo...,1,1,1/1,No,Followed,2019-09-30,/What-is-the-procedure-for-registration-for-Mo...,"Go to Mood Indigo, IIT Bombay | Asia's Largest...",0.5411,Positive,4,0.010870
767,767,Indian-Institute-of-Technology-Bombay-IITB,Is it possible to participate in the Techfest ...,4,1,4/1,Yes,Followed,2019-09-29,/Is-it-possible-to-participate-in-the-Techfest...,"Yes, Techfest conducts zonals to increase its ...",0.8679,Positive,5,0.043478
768,768,Indian-Institute-of-Technology-Bombay-IITB,What is the eligibility for participating in T...,1,1,1/1,No,Followed,2019-09-27,/What-is-the-eligibility-for-participating-in-...,To participate in IIT Bombay Techfest Robowars...,0.8271,Positive,5,0.010870
769,769,Indian-Institute-of-Technology-Bombay-IITB,What is the closing and opening rank of IIT Ka...,1,1,1/1,No,Followed,2019-09-25,/What-is-the-closing-and-opening-rank-of-IIT-K...,Search on Google www.jossa.ac.in,0.0000,Neutral,3,0.010870


In [21]:
engagement_median = df['followers_n'].median()
df["engagement"] = df['followers_n'].apply(lambda x: 1 if x >= engagement_median else 0)

In [22]:
def assignpred(n):
    if n==2:
        return 1
    if n==4:
        return 5
    return n

In [24]:
df['sentiment_score']=df['score']
df_eng1=df[df.engagement==1]
#if engagement score is greater than the median, increase the positive and negative sentiment
df_eng1['sentiment_score']=df_eng1['score'].apply(lambda x: assignpred(x))
df[df.engagement==1]=df_eng1
df

C:\Users\Kirthika\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Unnamed: 0,Topic,question,Followers,Answers,Ratio,Engagement potential,Last action,Parsed time,Question link,answers,compound,Sentiment,score,followers_n,engagement,sentiment_score
0,0,Indian-Institute-of-Technology-Bombay-IITB,Do only the students who start preparation for...,5,1,5/1,Yes,Followed,2020-07-22,/Do-only-the-students-who-start-preparation-fo...,Well actually not….though I am a NEET aspirant...,0.8388,Positive,5,0.054348,1,5
1,1,Indian-Institute-of-Technology-Bombay-IITB,How could I get into IIT Bombay? Is it possibl...,1,1,1/1,No,Followed,2020-07-21,/How-could-I-get-into-IIT-Bombay-Is-it-possibl...,"2 months and IIT Bombay, well umm what to say ...",0.5423,Positive,4,0.010870,0,4
2,2,Indian-Institute-of-Technology-Bombay-IITB,Can Kalpit Veerwal stop begging for ratings in...,15,1,15/1,Yes,Followed,2020-07-21,/Can-Kalpit-Veerwal-stop-begging-for-ratings-i...,"Hi, Nobody except Kalpit should be worried abo...",-0.3164,Negative,2,0.163043,1,1
3,3,Indian-Institute-of-Technology-Bombay-IITB,What are the subjects in aerospace engineering...,2,1,2/1,No,Followed,2020-07-20,/What-are-the-subjects-in-aerospace-engineerin...,"For such quarries in future as well, please vi...",0.7351,Positive,5,0.021739,1,5
4,4,Indian-Institute-of-Technology-Bombay-IITB,What is the full form of KGP?,2,1,2/1,No,Followed,2020-07-20,/What-is-the-full-form-of-KGP,"Knibb, Gormezano, & Partners Kannada Ganaka Pa...",-0.5423,Negative,2,0.021739,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
766,766,Indian-Institute-of-Technology-Bombay-IITB,What is the procedure for registration for Moo...,1,1,1/1,No,Followed,2019-09-30,/What-is-the-procedure-for-registration-for-Mo...,"Go to Mood Indigo, IIT Bombay | Asia's Largest...",0.5411,Positive,4,0.010870,0,4
767,767,Indian-Institute-of-Technology-Bombay-IITB,Is it possible to participate in the Techfest ...,4,1,4/1,Yes,Followed,2019-09-29,/Is-it-possible-to-participate-in-the-Techfest...,"Yes, Techfest conducts zonals to increase its ...",0.8679,Positive,5,0.043478,1,5
768,768,Indian-Institute-of-Technology-Bombay-IITB,What is the eligibility for participating in T...,1,1,1/1,No,Followed,2019-09-27,/What-is-the-eligibility-for-participating-in-...,To participate in IIT Bombay Techfest Robowars...,0.8271,Positive,5,0.010870,0,5
769,769,Indian-Institute-of-Technology-Bombay-IITB,What is the closing and opening rank of IIT Ka...,1,1,1/1,No,Followed,2019-09-25,/What-is-the-closing-and-opening-rank-of-IIT-K...,Search on Google www.jossa.ac.in,0.0000,Neutral,3,0.010870,0,3


In [26]:
df.rename(columns = {'score':'old_score'}, inplace = True) 

In [27]:
df.to_csv('../../Datasets/Quora/IITBQuoraWSA.csv')